In [ ]:
from google.colab import files

uploaded = files.upload()

Saving ratings.csv to ratings.csv


In [ ]:
import pandas as pd
import numpy as np
import io
from collections import Counter
from sklearn.model_selection import train_test_split
from scipy import sparse

In [ ]:
data = pd.read_csv(io.BytesIO(uploaded['ratings.csv']))

In [ ]:
data

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [ ]:
data.head(300)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
295,3,7899,4.5,1306464036
296,3,7991,5.0,1306463684
297,3,26409,4.5,1306463993
298,3,70946,5.0,1306463815


In [ ]:
Counter(data.rating)

Counter({0.5: 1370,
         1.0: 2811,
         1.5: 1791,
         2.0: 7551,
         2.5: 5550,
         3.0: 20047,
         3.5: 13136,
         4.0: 26818,
         4.5: 8551,
         5.0: 13211})

In [ ]:

Counter(data.groupby(['userId']).count()['movieId'])

Counter({20: 14,
         21: 15,
         22: 14,
         23: 13,
         24: 7,
         25: 9,
         26: 13,
         27: 8,
         28: 8,
         29: 8,
         30: 3,
         31: 7,
         32: 7,
         33: 10,
         34: 9,
         35: 11,
         36: 9,
         37: 6,
         38: 7,
         39: 4,
         40: 6,
         41: 4,
         42: 5,
         43: 3,
         44: 3,
         45: 6,
         46: 5,
         47: 4,
         48: 7,
         50: 7,
         51: 5,
         52: 2,
         53: 4,
         54: 3,
         55: 2,
         56: 14,
         57: 4,
         58: 5,
         59: 3,
         60: 2,
         61: 5,
         62: 2,
         63: 1,
         64: 5,
         65: 4,
         66: 2,
         67: 2,
         68: 1,
         69: 5,
         70: 2,
         71: 1,
         72: 2,
         73: 2,
         74: 2,
         75: 3,
         76: 3,
         77: 3,
         78: 3,
         80: 1,
         81: 2,
         82: 2,
         83: 3,


In [ ]:
np.mean(data.groupby(['userId']).count()['movieId'])

165.30491803278687

In [ ]:
train_df, valid_df = train_test_split(data, test_size=0.2)

In [ ]:
train_df = train_df.reset_index()[['userId', 'movieId', 'rating']]
valid_df = valid_df.reset_index()[['userId', 'movieId', 'rating']]

In [ ]:
train_df

,userId,movieId,rating
0,33,21,4.0
1,288,838,3.0
2,414,475,5.0
3,298,3967,2.0
4,316,3186,3.5
...,...,...,...
80663,272,1682,4.0
80664,314,327,1.0
80665,103,2712,4.5
80666,221,6662,4.0


In [ ]:
valid_df

,userId,movieId,rating
0,102,349,4.0
1,452,1687,5.0
2,318,2360,4.5
3,540,1971,2.5
4,414,2694,3.0
...,...,...,...
20163,376,10,4.5
20164,379,589,4.0
20165,525,4306,3.5
20166,599,112183,3.5


In [ ]:
def encode_column(column):
    """ Encodes a pandas column with continous IDs"""
    keys = column.unique()
    key_to_id = {key:idx for idx,key in enumerate(keys)}
    return key_to_id, np.array([key_to_id[x] for x in column]), len(keys)

In [ ]:
def encode_df(data):
    """Encodes rating data with continuous user and anime ids"""
    
    movie_ids, data['movieId'], num_movie = encode_column(data['movieId'])
    user_ids, data['userId'], num_users = encode_column(data['userId'])
    return data, num_users, num_movie, user_ids, movie_ids

In [ ]:
data, num_users, num_movie, user_ids, movie_ids = encode_df(train_df)
print("Number of users :", num_users)
print("Number of anime :", num_movie)
data.head()

Number of users : 610
Number of anime : 9724


,userId,movieId,rating,timestamp
0,0,0,4.0,964982703
1,0,1,4.0,964981247
2,0,2,4.0,964982224
3,0,3,5.0,964983815
4,0,4,5.0,964982931


In [ ]:
data

,userId,movieId,rating,timestamp
0,0,0,4.0,964982703
1,0,1,4.0,964981247
2,0,2,4.0,964982224
3,0,3,5.0,964983815
4,0,4,5.0,964982931
...,...,...,...,...
100831,609,3120,4.0,1493848402
100832,609,2035,5.0,1493850091
100833,609,3121,5.0,1494273047
100834,609,1392,5.0,1493846352


In [ ]:
def create_embeddings(n, K):
    """
    Creates a random numpy matrix of shape n, K with uniform values in (0, 11/K)
    n: number of items/users
    K: number of factors in the embedding 
    """
    return 11*np.random.random((n, K)) / K

In [ ]:
def create_sparse_matrix(df, rows, cols, column_name="rating"):
    """ Returns a sparse utility matrix""" 
    return sparse.csc_matrix((df[column_name].values,(df['userId'].values, df['movieId'].values)),shape=(rows, cols))

In [ ]:
data, num_users, num_movie, user_ids, movie_ids = encode_df(train_df)
Y = create_sparse_matrix(data, num_users, num_movie)

In [ ]:
num_movie

9724

In [ ]:
Y.todense()

matrix([[4. , 4. , 4. , ..., 0. , 0. , 0. ],
        [0. , 0. , 0. , ..., 0. , 0. , 0. ],
        [0. , 0. , 0. , ..., 0. , 0. , 0. ],
        ...,
        [2.5, 2. , 0. , ..., 0. , 0. , 0. ],
        [3. , 0. , 0. , ..., 0. , 0. , 0. ],
        [5. , 0. , 5. , ..., 3. , 3.5, 3.5]])

In [ ]:
def predict(df, emb_user, emb_movie):
    """ This function computes df["prediction"] without doing (U*V^T).
    
    Computes df["prediction"] by using elementwise multiplication of the corresponding embeddings and then 
    sum to get the prediction u_i*v_j. This avoids creating the dense matrix U*V^T.
    """
    df['prediction'] = np.sum(np.multiply(emb_movie[df['movieId']],emb_user[df['userId']]), axis=1)
    return df

In [ ]:
def cost(df, emb_user, emb_movie):
    """ Computes mean square error"""
    Y = create_sparse_matrix(df, emb_user.shape[0], emb_movie.shape[0])
    predicted = create_sparse_matrix(predict(df, emb_user, emb_movie), emb_user.shape[0], emb_movie.shape[0], 'prediction')
    return np.sum((Y-predicted).power(2))/df.shape[0]

In [ ]:
lmbda = 0.0002
def gradient(df, emb_user, emb_movie):
    """ Computes the gradient for user and anime embeddings"""
    Y = create_sparse_matrix(df, emb_user.shape[0], emb_movie.shape[0])
    predicted = create_sparse_matrix(predict(df, emb_user, emb_movie), emb_user.shape[0], emb_movie.shape[0], 'prediction')
    delta =(Y-predicted)
    grad_user = (-2/df.shape[0])*(delta*emb_movie) + 2*lmbda*emb_user
    grad_movie = (-2/df.shape[0])*(delta.T*emb_user) + 2*lmbda*emb_movie
    return grad_user, grad_movie

In [ ]:
def gradient_descent(df, emb_user, emb_movie, iterations=2000, learning_rate=0.01, df_val=None):
    """ 
    Computes gradient descent with momentum (0.9) for given number of iterations.
    emb_user: the trained user embedding
    emb_anime: the trained anime embedding
    """
    Y = create_sparse_matrix(df, emb_user.shape[0], emb_movie.shape[0])
    beta = 0.9
    grad_user, grad_movie = gradient(df, emb_user, emb_movie)
    v_user = grad_user
    v_movie = grad_movie
    for i in range(iterations):
        grad_user, grad_movie = gradient(df, emb_user, emb_movie)
        v_user = beta*v_user + (1-beta)*grad_user
        v_movie = beta*v_movie + (1-beta)*grad_movie
        emb_user = emb_user - learning_rate*v_user
        emb_movie = emb_movie - learning_rate*v_movie
        if(not (i+1)%50):
            print("\niteration", i+1, ":")
            print("train mse:",  cost(df, emb_user, emb_movie))
            if df_val is not None:
                print("validation mse:",  cost(df_val, emb_user, emb_movie))
    return emb_user, emb_movie

In [ ]:
emb_user = create_embeddings(num_users, 3)
emb_movie = create_embeddings(num_movie, 3)
emb_user, emb_movie = gradient_descent(data, emb_user, emb_movie, iterations=800, learning_rate=1)


iteration 50 :
train mse: 5.056022536249493

iteration 100 :
train mse: 3.289468719070871

iteration 150 :
train mse: 2.613344417374374

iteration 200 :
train mse: 2.2236954509546156

iteration 250 :
train mse: 1.965755218180462

iteration 300 :
train mse: 1.7808267723880815

iteration 350 :
train mse: 1.6413109011833664

iteration 400 :
train mse: 1.5322989755099927

iteration 450 :
train mse: 1.4449001839234288

iteration 500 :
train mse: 1.3734143343708516

iteration 550 :
train mse: 1.3139862912567124

iteration 600 :
train mse: 1.2639029732611926

iteration 650 :
train mse: 1.2211967682867597

iteration 700 :
train mse: 1.1844071590150393

iteration 750 :
train mse: 1.1524298018121988

iteration 800 :
train mse: 1.1244168686020837


In [ ]:
def encode_new_data(valid_df, user_ids, movie_ids):
    """ Encodes valid_df with the same encoding as train_df.
    """
    df_val_chosen = valid_df['movieId'].isin(movie_ids.keys()) & valid_df['userId'].isin(user_ids.keys())
    valid_df = valid_df[df_val_chosen]
    valid_df['movieId'] =  np.array([movie_ids[x] for x in valid_df['movieId']])
    valid_df['userId'] = np.array([user_ids[x] for x in valid_df['userId']])
    return valid_df

In [ ]:
print("before encoding:", valid_df.shape)
valid_df = encode_new_data(valid_df, user_ids, movie_ids)
print("after encoding:", valid_df.shape)

before encoding: (20168, 3)
after encoding: (15489, 3)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
val_mse = cost(valid_df, emb_user, emb_movie)
print(val_mse)

2.25047776829391


In [ ]:
valid_df[70:80].head()

,userId,movieId,rating,prediction
87,418,593,4.0,2.661135
89,571,2149,1.0,6.433659
91,42,1500,5.0,4.598168
92,33,50,5.0,2.709864
93,325,3770,4.0,5.528699


In [ ]:
valid_df

,userId,movieId,rating,prediction
0,220,4011,4.0,4.978092
1,483,2959,4.5,3.060584
2,504,1968,4.0,3.559915
3,480,6754,3.5,3.269896
4,438,6708,3.5,2.889526
...,...,...,...,...
20161,489,2124,2.5,1.746680
20163,156,1948,3.0,0.967712
20164,422,1617,4.0,4.928762
20166,525,2916,3.5,3.767881
